In [1]:
import os
import shutil
import glob
import random
import warnings
import subprocess
import concurrent.futures
import multiprocessing
from itertools import chain
from collections import defaultdict, OrderedDict
from functools import partial
from scipy import stats, sparse
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import anndata
import networkx as nx
import scanpy as sc
import seaborn as sns
import h5py
import scglue
import multiprocessing
import matplotlib.pyplot as plt
from lisa.core.data_interface import DataInterface
from lisa import FromCoverage, FromGenes, FromRegions
from lisa.core.assays import LISA_RP_Assay, delta_RP_wrapper, transform_RP,get_delta_RP,get_deltaRP_activation
from supercell import *
from Ori_data import *
from utils import *
warnings.filterwarnings("ignore")

/fs/home/xuyunfan/miniconda3/envs/scrip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Data

## Obtain the supercell division of each cell from RNA data

In [2]:
atac = sc.read_h5ad('/fs/home/xuyunfan/project/scripmulti/mira/atac_scrip.h5ad')

In [ ]:
rna = sc.read_h5ad("/fs/home/xuyunfan/data/mira/shareseq_base_data/rna_data.h5ad")

In [ ]:
rna.var_names_make_unique()
rna.var_names = [x.capitalize() for x in rna.var_names]

In [ ]:
test_data = Ori_Data(rna,Cell_num=50)

In [ ]:
%%time
test_data.get_positive_marker_gene_parallel(cores=4)

In [ ]:
process_and_merge(atac_df, './', n_num=20)

In [ ]:
cellgroup = test_data.adata.obs.loc[:,['new_leiden']]

## Obtain the landscape of each supercell.

In [ ]:
atac_df=atac.to_df()

In [ ]:
get_supercell_fragment(cellgroup,'.','./merge.tsv',chunksize = 10000000)

In [ ]:
process_tsv('./supercell_fragment/', '~/data/bedGraphToBigWig','~/data/mm10.genome')

# Calculate TF score

In [ ]:
share_seq = SCRIPro_Multiome(8,'mm10',test_data)

In [ ]:
%%time
share_seq.cal_ISD_parallel('./bigwig/')

In [ ]:
share_seq.get_P_value_matrix()

In [ ]:
share_seq.get_chip_matrix()

# Calculate TF targets

In [ ]:
target_h5=pd.HDFStore('~/project/scripmulti/TTTTT/atac_rna/ALL_target_3.h5')

In [ ]:
share_seq.get_tf(target_h5)

In [ ]:
share_seq.tf_score

In [ ]:
share_seq.get_tf_target('Gata3')